In [51]:
from pysentimiento import create_analyzer
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
# from pysentimiento.preprocessing import preprocess_tweet # using github preprocess instead
import preprocessor as p
import spacy
import tweepy
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

b_analyzer_sentiment = create_analyzer(task="sentiment", lang="en")
b_analyzer_emotion = create_analyzer(task="emotion", lang="en")
b_analyzer_hate = create_analyzer(task="hate_speech", lang="en")
v_analyzer_sentiment = SentimentIntensityAnalyzer()
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG)
TweetTokenizer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

loading configuration file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json from cache at C:\Users\Joe Wilson/.cache\huggingface\transformers\cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_he

Realtime Tweet Sentiment Analyzer using BERTWEET engine and customized ML

In [52]:
def tweet_sentiment_analyzer(text):
    
    temp = b_analyzer_sentiment.predict(text)
    bert_sentiment_label = temp.output # output = neg, pos, neu label
    bert_sentiment_score = temp.probas # probas = percentage score
    NEG = round(bert_sentiment_score.get('POS'), 2)
    POS = round(bert_sentiment_score.get('NEG'), 2)
    NEU = round(bert_sentiment_score.get('NEU'), 2)
    score_probability = max(NEG, POS, NEU)
    bert_sentiment_list = [bert_sentiment_label, score_probability]
    
    return(bert_sentiment_list)

Realtime Tweet Tokenizer using nltk's tweet tokenizer

In [53]:
def tweet_tokenization(clean_tweet_text):
    
    token = TweetTokenizer()
    temp_tokens = token.tokenize(clean_tweet_text)  
    nltk_tokens = []
    for w in temp_tokens:
         if w not in stop_words:
             if len(w) > 2:
                 nltk_tokens.append(w)
                 
    return(nltk_tokens)

In [20]:
def tweet_emotion_analyzer(text):
    
 temp = b_analyzer_emotion.predict(text)
 label = temp.output
 
 return (label)

In [46]:
def tweet_hate_analyzer(text):
    
     temp = b_analyzer_hate.predict(text)
     temp_label = temp.output
     if len(temp_label) > 2:
          a = temp_label[0]
          b = temp_label[1]
          c = temp_label[2]
          h = (a + " " + b + " " + c)
     elif len(temp_label) == 2:
          a = temp_label[0]
          b = temp_label[1]
          h = (a + " " + b)
          
     elif len(temp_label) == 1:
          h = temp_label[0]
     else:
          h = "None"
     
     return (h)

In [57]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')

query = "republicans -is:retweet lang:en"
# response = client.search_recent_tweets(query=query, tweet_fields=['author_id'], user_fields=['created_at', 'name'],
#                               max_results=10)

response = client.search_recent_tweets(query=query,tweet_fields=['attachments','author_id','context_annotations','conversation_id','created_at','entities','geo,id','in_reply_to_user_id','lang','possibly_sensitive','public_metrics','referenced_tweets','reply_settings','source','text','withheld'],user_fields=['created_at','description','entities,id','location','name','pinned_tweet_id','profile_image_url','protected,public_metrics','url','username','verified','withheld'],expansions=['attachments.poll_ids','attachments.media_keys','author_id','geo.place_id','in_reply_to_user_id','referenced_tweets.id','entities.mentions.username','referenced_tweets.id.author_id'],media_fields=['duration_ms','height','media_key', 'preview_image_url','promoted_metrics','public_metrics','type,url'],place_fields=['contained_within,country','country_code','full_name','geo,id','name','place_type'],poll_fields=['duration_minutes','end_datetime','id','options','voting_status'],max_results=10) 

for tweet in response.data: 
    # print("orginal tweet: " + tweet.text)
    print('----------------------------------------------------------------------------------------')
    clean_tweet_text = p.clean(tweet.text)
    clean_tweet_text = clean_tweet_text.replace("&amp", "")  
    tweet_sentiment = tweet_sentiment_analyzer(tweet.text)
    nltk_tokens = tweet_tokenization(clean_tweet_text) 
    tweet_emotion_label = tweet_emotion_analyzer(tweet.text)
    tweet_hate_label = tweet_hate_analyzer(tweet.text)
    # print(clean_tweet_text)
    # print(tweet_sentiment)
    # print(tweet_emotion_label)
    # print(tweet_hate_label)
    
    print(type(tweet.context_annotations))
    print(len(tweet.context_annotations))
    print('------------------------------------------------------------------------------------------')

----------------------------------------------------------------------------------------
<class 'list'>
3
------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------
<class 'list'>
0
------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------
<class 'list'>
2
------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------
<class 'list'>
0
------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------
<class 'list'>
2
------------------------------------------------------------------------------------------
---------------